## 피마 인디언 당뇨병 예측

피마 인디언 당뇨병(Pima Indian Diabetes) 데이터 세트를 이용해  
당뇨병 여부를 판단하는 머신러닝 예측 모델을 수립하고  
지금까지 학습한 오차 행렬, 정확도, 정밀도, 재현율, F1 스코더, AUC 평가 지표 적용

https://www.kaggle.com/  
회원가입하고 데이터 다운로드  
https://www.kaggle.com/uciml/pima-indians-diabetes-database 데이터 다운로드  
diabetes.csv

피마 인디언 당뇨병 데이터 세트 피처
* Pregnancies: 임신 횟수
* Glucose: 포도당 부하 검사 수치
* BloodPressure: 혈압(mm Hg)
* SkinThickness: 팔 삼두근 뒤쪽의 피하지방 측정값(mm)
* Insulin: 혈청 인슐린(mu U/ml)
* BMI: 체질량지수(체중(kg)/(키(m))^2)
* DiabetesPedigreeFunction: 당뇨 내력 가중치 값
* Age: 나이
* Outcome: 클래스 결정 값(0또는 1)

In [ ]:
# 데이터 정보 확인


로지스틱 회귀를 이용해 예측 모델 생성
- 데이터 세트를 피처 데이터 세트와 클래스 데이터 세트로 나누고
- 학습 데이터 세트와 테스트 데이터 세트로 분리
- 로지스틱 회귀를 이용해 예측 수행
- 성능 평가 지표 출력
- 재현율 곡선 시각화

**앞 예제에서 사용된 get_clf_eval()과 precision_recall_curve_plot() 재 로딩**

In [ ]:
# 수정된 get_clf_eval() 함수 


In [ ]:
# 정밀도-재현율 커브 플롯


**Logistic Regression으로 학습 및 예측 수행**

In [ ]:
# 피처 데이터세트 X, 레이블 데이터세트 y


# 학습 데이터 세트와 테스트 데이터 세트로 분리


# 로지스틱 회귀로 학습,예측 및 평가


# 예측 확률 : roc_auc_score 용 



## 재현율 성능에 조금 더 초점을 맞춰 조정

정밀도와 재현율 곡선을 확인 

**각 피처들의 값 분포 확인**

- Glucose(포도당) 피처의 히스토그램

**0값이 있는 피처들에서 0값의 데이터 건수와 퍼센트 계산**

In [ ]:
# 0값을 검사할 피처명 리스트 객체 설정


In [ ]:
# 전체 데이터 건수


In [ ]:
# 피처별로 반복 하면서 데이터 값이 0 인 데이터 건수 추출하고, 퍼센트 계산


**0값을 평균값으로 대체**

- 0값 검사

**StandardScaler 클래스를 이용해 스케일링 적용 학습/예측**

In [ ]:
# 피처, 레이블 데이터 세트 지정


# 표준화 스케일링


# 학습/테스트 데이터 세트로 분리



# 로지스틱 회귀로 학습, 예측 및 평가 수행. 



# 성능 지표 출력




**분류결정 임계값을 변화시키면서 재현율 성능 측정**
- 임계값을 0.3에서 0.5까지 0.03씩 변화


In [ ]:
# 임계값에 따른 평가 수치 출력 함수 get_eval_by_threshold()



**임계값을 0.48로 설정하여 예측한 성능 지표 확인**